## Introduction / Business Problem:
### Business Understanding: 
Be Our Guest; one of the biggest Restaurant groups in the U.S, getting incredible attention, awareness and popularity with one of their restaurants in Chicago. Bill’s Chicago’s popularity; which is in Downtown Chicago and known for their award wining burgers and craft beers is expanding outside of Chicago and Be Our Guest is looking to open another restaurant in New York City with the intention to generate more revenue for the company. 
### Description of the problem:
For Be Our Guest Stakeholders, the problem that is in question is that “What is the best area to open a restaurant in New York City for Bill’s Chicago Burgers.” The main goal is to understand the NYC restaurant market and improve probability of success and business profitability. Besides generating revenue, Be Our Guest Stakeholders are looking to improve the awareness of “Bills’ Chicago” ’s brand. 
Even though, Be Our Guest Restaurant group has many restaurants in multiple different categories in New York City, developing a new restaurant requires certain investment and research to ensure the company’s profitability and defining the restaurant location is the most important criteria when it comes to success of a restaurant. 
### Analytical Approach:
We will be looking at the relationships of different variables and probability of success. So, we will be using a descriptive and predictive analytics as part of our data analysis.
### Data Requirements:
Since the solution to our problem requires a location information, we need city, borough, neighborhood information. We also need the venue and category information within these locations so we can access the best possible area to open up a restaurant in NYC. 

### Data Collection:
Data Collection:
Essentially, we will be using a dataset that covers all the 5 boroughs, the neighborhoods that exists in each borough, latitude and longitude coordinates.  This data set is available to use from NYU website - https://geo.nyu.edu/catalog/nyu_2451_34572 - (This dataset is also available to be run  with a wget command via json here: https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json 
We will be using this dataset to get the borough, neighborhood, latitude and longitude coordinates. 
We will also use Foursquare developer (personal account) api to collect certain variables such as existing restaurants, restaurant categories, locations (latitude and longitude), nearby venue list in each borough and unique categories. This will allow us to see the competition, the restaurant marketplace insights and possible relationship between venue category (such as Burger Restaurant) , borough and / or neighborhood. 
#### Let's get started with importing all the required libraries.

In [14]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


#### Let's download our dataset and start exploring.

In [15]:
!wget -q -O 'newyork_data.json' https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json
print('Data downloaded!')

Data downloaded!


In [16]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [17]:
newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

There is a relationship within the features data. Let's define a new variable "neighborhoods" with this features data.

### Data Understanding and Preparation: 
By exploring the datasets from NYU and Foursquare, we will create our data frame and perform exploration to further understand variables and possible correlations. Based on our understandings we clean up the data frame so missing data values are updated or ignored, neighborhood is broken down within Boroughs, create visualizations to help us understand to further segment and cluster the neighborhoods in a certain Borough. 
We also leverage the data we get from Foursquare api around venue category type, possible analyzation of each neighborhood for common venues to see if there are any gaps that Bill’s Chicago can fill in terms of type of food (Burger Restaurant) 

In [18]:
neighborhoods_data = newyork_data['features']

In [19]:
neighborhoods_data[1]

{'type': 'Feature',
 'id': 'nyu_2451_34572.2',
 'geometry': {'type': 'Point',
  'coordinates': [-73.82993910812398, 40.87429419303012]},
 'geometry_name': 'geom',
 'properties': {'name': 'Co-op City',
  'stacked': 2,
  'annoline1': 'Co-op',
  'annoline2': 'City',
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.82993910812398,
   40.87429419303012,
   -73.82993910812398,
   40.87429419303012]}}

#### Let's create our dataframe

From this dataset as planned we want to use Borough, Neighborhood, Latitude and Longitude as information. So let's set these as our columns.

In [20]:
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude']

In [21]:
neighborhoods = pd.DataFrame(columns=column_names)

In [22]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude


Empty. Let's fill in the dataframe , one row at a time from our dataset.

In [23]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [24]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


We only see Bronx and dont want to run the entire dataframe to examine. Let's see if we captured all the Boroughs.

In [25]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


#### Let's get the map of NYC and add our neighborhoods.

In [26]:
address = 'New York City, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of New York City are 40.7308619, -73.9871558.


In [27]:
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

We have all the Boroughs now. We want to narrow down to one Borough. Brooklyn is trending for their comfort food. So Let's explore Brooklyn.

In [28]:
brooklyn_data = neighborhoods[neighborhoods['Borough'] == 'Brooklyn'].reset_index(drop=True)
brooklyn_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Brooklyn,Bay Ridge,40.625801,-74.030621
1,Brooklyn,Bensonhurst,40.611009,-73.995180
2,Brooklyn,Sunset Park,40.645103,-74.010316
3,Brooklyn,Greenpoint,40.730201,-73.954241
4,Brooklyn,Gravesend,40.595260,-73.973471


In [29]:
address = 'Brooklyn, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Brooklyn are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Brooklyn are 40.6501038, -73.9495823.


In [31]:
map_brooklyn = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, label in zip(brooklyn_data['Latitude'], brooklyn_data['Longitude'], brooklyn_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_brooklyn)  
    
map_brooklyn

We gathered all the location information for NYC, We are specifically researching for Brooklyn location now. We can start looking into the venues and marketplace. We need to leverage and use Foursquare for this.

In [32]:
CLIENT_ID = 'K0J1ZEUIPQF524H4YMTSDQX034ODNSAZR5F3ZJK2SBCRT5PA'
CLIENT_SECRET = 'EPQ1S4G3TRW5THSZLW3MBTYOG1RHVCKBOCAGYE1G0SE1PLHV'
VERSION = '20180605' 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: K0J1ZEUIPQF524H4YMTSDQX034ODNSAZR5F3ZJK2SBCRT5PA
CLIENT_SECRET:EPQ1S4G3TRW5THSZLW3MBTYOG1RHVCKBOCAGYE1G0SE1PLHV


In [38]:
brooklyn_data.loc[3, 'Neighborhood']

'Greenpoint'

In [41]:
neighborhood_latitude = brooklyn_data.loc[3, 'Latitude']
neighborhood_longitude = brooklyn_data.loc[3, 'Longitude']

neighborhood_name = brooklyn_data.loc[3, 'Neighborhood']

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Greenpoint are 40.7302009848647, -73.95424093127393.


Let's look at the venues around Greenpoint.

In [42]:
LIMIT = 100 
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=K0J1ZEUIPQF524H4YMTSDQX034ODNSAZR5F3ZJK2SBCRT5PA&client_secret=EPQ1S4G3TRW5THSZLW3MBTYOG1RHVCKBOCAGYE1G0SE1PLHV&v=20180605&ll=40.7302009848647,-73.95424093127393&radius=500&limit=100'

In [43]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c20ef12dd579735ee8a8e58'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Greenpoint',
  'headerFullLocation': 'Greenpoint, Brooklyn',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 114,
  'suggestedBounds': {'ne': {'lat': 40.734700989364704,
    'lng': -73.94831369618184},
   'sw': {'lat': 40.7257009803647, 'lng': -73.96016816636602}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4a9a86dcf964a520cc3120e3',
       'name': 'goodyoga',
       'location': {'address': '114 Greenpoint Ave',
        'crossStreet': 'btwn Franklin Ave & Manhattan Ave',
        'lat': 40.7300096443983,
      

Let's get the type of venues in Greenpoint.

In [44]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Let's create the data frame for the venues around Greenpoint.

In [45]:
venues = results['response']['groups'][0]['items']    
nearby_venues = json_normalize(venues)
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,goodyoga,Yoga Studio,40.730010,-73.956167
1,Sunshine Laundry & Pinball Emporium,Laundry Service,40.729318,-73.953564
2,Karczma,Polish Restaurant,40.730102,-73.955092
3,Oxomoco,Mexican Restaurant,40.729981,-73.955460
4,Early,Café,40.732069,-73.954721


In [46]:
nearby_venues.head(100)

,name,categories,lat,lng
0,goodyoga,Yoga Studio,40.730010,-73.956167
1,Sunshine Laundry & Pinball Emporium,Laundry Service,40.729318,-73.953564
2,Karczma,Polish Restaurant,40.730102,-73.955092
3,Oxomoco,Mexican Restaurant,40.729981,-73.955460
4,Early,Café,40.732069,-73.954721
5,Black Rabbit,Bar,40.730057,-73.956588
6,Fornino,Pizza Place,40.728951,-73.953862
7,The Garden,Grocery Store,40.730760,-73.954502
8,Friducha,Mexican Restaurant,40.731512,-73.954281
9,Pelicana Chicken,Fried Chicken Joint,40.731452,-73.954510


In [47]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


As planned we are looking at 100 venues within Greenpoint. 

We have only been looking at Greenpoint as a neighborhood so far. Let's look at other neighborhoods in Brooklyn.

In [48]:
def getNearbyVenues(names, latitudes, longitudes, radius=500): 
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [49]:
brooklyn_venues = getNearbyVenues(names=brooklyn_data['Neighborhood'],
                                   latitudes=brooklyn_data['Latitude'],
                                   longitudes=brooklyn_data['Longitude']
                                  )

Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker Heights
Gerritsen Beach
Marine Park
Clinton Hill
Sea Gate
Downtown
Boerum Hill
Prospect Lefferts Gardens
Ocean Hill
City Line
Bergen Beach
Midwood
Prospect Park South
Georgetown
East Williamsburg
North Side
South Side
Ocean Parkway
Fort Hamilton
Ditmas Park
Wingate
Rugby
Remsen Village
New Lots
Paerdegat Basin
Mill Basin
Fulton Ferry
Vinegar Hill
Weeksville
Broadway Junction
Dumbo
Homecrest
Highland Park
Madison
Erasmus


In [50]:
print(brooklyn_venues.shape)
brooklyn_venues.head()

(2819, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bay Ridge,40.625801,-74.030621,Pilo Arts Day Spa and Salon,40.624748,-74.030591,Spa
1,Bay Ridge,40.625801,-74.030621,Bagel Boy,40.627896,-74.029335,Bagel Shop
2,Bay Ridge,40.625801,-74.030621,Cocoa Grinder,40.623967,-74.030863,Juice Bar
3,Bay Ridge,40.625801,-74.030621,Pegasus Cafe,40.623168,-74.031186,Breakfast Spot
4,Bay Ridge,40.625801,-74.030621,Ho' Brah Taco Joint,40.622960,-74.031371,Taco Place


In [51]:
brooklyn_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bath Beach,53,53,53,53,53,53
Bay Ridge,87,87,87,87,87,87
Bedford Stuyvesant,27,27,27,27,27,27
Bensonhurst,31,31,31,31,31,31
Bergen Beach,7,7,7,7,7,7
Boerum Hill,89,89,89,89,89,89
Borough Park,21,21,21,21,21,21
Brighton Beach,43,43,43,43,43,43
Broadway Junction,14,14,14,14,14,14


We can easily see the amount of venues against each neighborhood within Brooklyn. Let's find how many unique categories we have in brooklyn.

In [53]:
print('There are {} uniques categories.'.format(len(brooklyn_venues['Venue Category'].unique())))

There are 288 uniques categories.


In [54]:
brooklyn_onehot = pd.get_dummies(brooklyn_venues[['Venue Category']], prefix="", prefix_sep="")
brooklyn_onehot['Neighborhood'] = brooklyn_venues['Neighborhood'] 
fixed_columns = [brooklyn_onehot.columns[-1]] + list(brooklyn_onehot.columns[:-1])
brooklyn_onehot = brooklyn_onehot[fixed_columns]

brooklyn_onehot.head()

,Yoga Studio,Adult Boutique,Airport Terminal,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,History Museum,Hobby Shop,Hockey Field,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Tiki Bar,Toy / Game Store,Trail,Turkish Restaurant,Used Bookstore,Vape Store,Varenyky

In [55]:
brooklyn_onehot.head(100)

,Yoga Studio,Adult Boutique,Airport Terminal,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,History Museum,Hobby Shop,Hockey Field,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Tiki Bar,Toy / Game Store,Trail,Turkish Restaurant,Used Bookstore,Vape Store,Varenyky

In [56]:
brooklyn_onehot.shape

(2819, 288)

Since we are analysizing each neighborhood, let's group the data frame by neighborhood.

In [57]:
brooklyn_grouped = brooklyn_onehot.groupby('Neighborhood').mean().reset_index()
brooklyn_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Airport Terminal,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,History Museum,Hobby Shop,Hockey Field,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Non-Profit,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Tiki Bar,Toy / Game Store,Trail,Turkish Restaurant,Used Bookstore,Vape Store,Varenyky

Let's see the top 10 venues

In [58]:
num_top_venues = 10

for hood in brooklyn_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = brooklyn_grouped[brooklyn_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bath Beach----
                  venue  freq
0              Pharmacy  0.06
1            Donut Shop  0.06
2      Sushi Restaurant  0.04
3    Chinese Restaurant  0.04
4    Italian Restaurant  0.04
5  Fast Food Restaurant  0.04
6           Pizza Place  0.04
7     Mobile Phone Shop  0.04
8            Kids Store  0.04
9       Bubble Tea Shop  0.02


----Bay Ridge----
                 venue  freq
0                  Spa  0.07
1   Italian Restaurant  0.07
2          Pizza Place  0.06
3           Bagel Shop  0.03
4  American Restaurant  0.03
5             Pharmacy  0.03
6      Thai Restaurant  0.03
7                  Bar  0.03
8     Greek Restaurant  0.03
9           Playground  0.02


----Bedford Stuyvesant----
                 venue  freq
0          Coffee Shop  0.11
1                  Bar  0.07
2          Pizza Place  0.07
3                 Café  0.07
4            Juice Bar  0.04
5        Deli / Bodega  0.04
6                 Park  0.04
7  Fried Chicken Joint  0.04
8    Other Repair Shop

Let's put them inthe dataframe and make them in order.

In [59]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [60]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = brooklyn_grouped['Neighborhood']
for ind in np.arange(brooklyn_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(brooklyn_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bath Beach,Donut Shop,Pharmacy,Pizza Place,Italian Restaurant,Sushi Restaurant,Kids Store,Mobile Phone Shop,Chinese Restaurant,Fast Food Restaurant,Coffee Shop
1,Bay Ridge,Spa,Italian Restaurant,Pizza Place,Greek Restaurant,Bar,Thai Restaurant,American Restaurant,Bagel Shop,Pharmacy,Hookah Bar
2,Bedford Stuyvesant,Coffee Shop,Bar,Pizza Place,Café,Bus Stop,Other Repair Shop,Thrift / Vintage Store,Bagel Shop,BBQ Joint,Gourmet Shop
3,Bensonhurst,Chinese Restaurant,Ice Cream Shop,Sushi Restaurant,Grocery Store,Pizza Place,Shabu-Shabu Restaurant,Bagel Shop,Supermarket,Coffee Shop,Noodle House
4,Bergen Beach,Harbor / Marina,Donut Shop,Athletics & Sports,Baseball Field,Hockey Field,Playground,Food & Drink Shop,Field,Filipino Restaurant,Fish & Chips Shop
5,Boerum Hill,Coffee Shop,Bar,French Restaurant,Dance Studio,Yoga Studio,Cosmetics Shop,Spa,Furniture / Home Store,Sandwich Place,Martial Arts Dojo
6,Borough Park,Pizza Place,Bank,Deli / Bodega,Hotel,Pharmacy,Fast Food Restaurant,Bike Shop,Coffee Shop,Paper / Office Supplies Store,Bakery
7,Brighton Beach,Eastern European Restaurant,Restaurant,Beach,Pharmacy,Fast Food Restaurant,Russian Restaurant,Sushi Restaurant,Mobile Phone Shop,Bank,Gourmet Shop
8,Broadway Junction,Diner,Donut Shop,Burger Joint,Moving Target,Fried Chicken Joint,Bus Station,Caribbean Restaurant,Seafood Restaurant,Sandwich Place,Deli / Bodega
9,Brooklyn Heights,Yoga Studio,Park,Italian Restaurant,Gym,Cosmetics Shop,Coffee Shop,Deli / Bodega,Thai Restaurant,Food Truck,Sushi Restaurant


### Modeling
We are going to segment the boroughs and neighborhoods using Clustering Method – (k-means) , we can group top 5-10 venues for each neighborhood and further examine each cluster to define the best possible Neighborhood / Borough for Bill’s Chicago.

In [61]:
kclusters = 5
brooklyn_grouped_clustering = brooklyn_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(brooklyn_grouped_clustering)
kmeans.labels_[0:10] 

array([3, 3, 3, 3, 3, 3, 3, 3, 0, 3], dtype=int32)

Let's create another dataframe that includes clusters.

In [62]:
brooklyn_merged = brooklyn_data
brooklyn_merged['Cluster Labels'] = kmeans.labels_
brooklyn_merged = brooklyn_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
brooklyn_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Brooklyn,Bay Ridge,40.625801,-74.030621,3,Spa,Italian Restaurant,Pizza Place,Greek Restaurant,Bar,Thai Restaurant,American Restaurant,Bagel Shop,Pharmacy,Hookah Bar
1,Brooklyn,Bensonhurst,40.611009,-73.995180,3,Chinese Restaurant,Ice Cream Shop,Sushi Restaurant,Grocery Store,Pizza Place,Shabu-Shabu Restaurant,Bagel Shop,Supermarket,Coffee Shop,Noodle House
2,Brooklyn,Sunset Park,40.645103,-74.010316,3,Bank,Bakery,Pizza Place,Latin American Restaurant,Mexican Restaurant,Gym,Pharmacy,Deli / Bodega,Mobile Phone Shop,Shoe Store
3,Brooklyn,Greenpoint,40.730201,-73.954241,3,Bar,Pizza Place,Café,Coffee Shop,Cocktail Bar,Yoga Studio,Record Shop,Boutique,Bakery,Mexican Restaurant
4,Brooklyn,Gravesend,40.595260,-73.973471,3,Chinese Restaurant,Lounge,Spa,Italian Restaurant,Pizza Place,Grocery Store,Donut Shop,Metro Station,Record Shop,Baseball Field


Let's visualize to see if we can see define findings on what neighborhood within Brooklyn is the place to open up a Burger shop.

In [63]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(brooklyn_merged['Latitude'], brooklyn_merged['Longitude'], brooklyn_merged['Neighborhood'], brooklyn_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Finally we can create and look at each cluster to solve the problem in question " Where should Be Our guest Open Bills' Chicago Burgers in NYC?"

In [64]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 0, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Flatbush,Mexican Restaurant,Juice Bar,Coffee Shop,Caribbean Restaurant,Bakery,Bagel Shop,Chinese Restaurant,Sandwich Place,Pizza Place,Middle Eastern Restaurant
12,Windsor Terrace,Deli / Bodega,Diner,Park,Plaza,Thrift / Vintage Store,Café,Food Truck,Moving Target,Chinese Restaurant,Salad Place
17,Bedford Stuyvesant,Coffee Shop,Bar,Pizza Place,Café,Bus Stop,Other Repair Shop,Thrift / Vintage Store,Bagel Shop,BBQ Joint,Gourmet Shop
20,Carroll Gardens,Italian Restaurant,Cocktail Bar,Coffee Shop,Bakery,Pizza Place,Wine Shop,Bar,Gym / Fitness Center,Spa,Gift Shop
24,Park Slope,Burger Joint,American Restaurant,Pizza Place,Coffee Shop,Italian Restaurant,Japanese Restaurant,Bakery,Bagel Shop,Pet Store,Pub
27,Starrett City,Moving Target,Bus Stop,Cosmetics Shop,Pizza Place,Bus Station,Supermarket,Shoe Store,Caribbean Restaurant,Gym,Gym Pool
28,Canarsie,Chinese Restaurant,Bus Line,Deli / Bodega,Gym,Caribbean Restaurant,Thai Restaurant,Asian Restaurant,Women's Store,Food & Drink Shop,Fish Market
29,Flatlands,Fast Food Restaurant,Pharmacy,Caribbean Restaurant,Fried Chicken Joint,Discount Store,Deli / Bodega,Lounge,Video Store,Bar,Electronics Store
56,Rugby,Supermarket,Caribbean Restaurant,Chinese Restaurant,Diner,Seafood Restaurant,Sandwich Place,Coffee Shop,Pizza Place,Deli / Bodega,Bank
58,New Lots,Pizza Place,Discount Store,Breakfast Spot,Fast Food Restaurant,Furniture / Home Store,Latin American Restaurant,Chinese Restaurant,Bank,Park,Asian Restaurant


In [65]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 1, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Cobble Hill,Bar,Pizza Place,Ice Cream Shop,Playground,Italian Restaurant,American Restaurant,Bakery,Yoga Studio,Cocktail Bar,Coffee Shop
25,Cypress Hills,Fast Food Restaurant,Pizza Place,Ice Cream Shop,Latin American Restaurant,Fried Chicken Joint,Donut Shop,Supermarket,Liquor Store,Mexican Restaurant,Food
48,Georgetown,Women's Store,Breakfast Spot,Donut Shop,Pharmacy,Bank,Shopping Mall,Supermarket,Burger Joint,Frozen Yogurt Shop,Shoe Store


In [66]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 2, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
47,Prospect Park South,Caribbean Restaurant,Fast Food Restaurant,Pizza Place,Grocery Store,Pharmacy,Mobile Phone Shop,Women's Store,Donut Shop,Clothing Store,Latin American Restaurant


In [67]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 3, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bay Ridge,Spa,Italian Restaurant,Pizza Place,Greek Restaurant,Bar,Thai Restaurant,American Restaurant,Bagel Shop,Pharmacy,Hookah Bar
1,Bensonhurst,Chinese Restaurant,Ice Cream Shop,Sushi Restaurant,Grocery Store,Pizza Place,Shabu-Shabu Restaurant,Bagel Shop,Supermarket,Coffee Shop,Noodle House
2,Sunset Park,Bank,Bakery,Pizza Place,Latin American Restaurant,Mexican Restaurant,Gym,Pharmacy,Deli / Bodega,Mobile Phone Shop,Shoe Store
3,Greenpoint,Bar,Pizza Place,Café,Coffee Shop,Cocktail Bar,Yoga Studio,Record Shop,Boutique,Bakery,Mexican Restaurant
4,Gravesend,Chinese Restaurant,Lounge,Spa,Italian Restaurant,Pizza Place,Grocery Store,Donut Shop,Metro Station,Record Shop,Baseball Field
5,Brighton Beach,Eastern European Restaurant,Restaurant,Beach,Pharmacy,Fast Food Restaurant,Russian Restaurant,Sushi Restaurant,Mobile Phone Shop,Bank,Gourmet Shop
6,Sheepshead Bay,Dessert Shop,Turkish Restaurant,Italian Restaurant,Sandwich Place,Restaurant,Department Store,Chinese Restaurant,Russian Restaurant,Creperie,Yoga Studio
7,Manhattan Terrace,Pizza Place,Cosmetics Shop,Ice Cream Shop,Donut Shop,Bagel Shop,Coffee Shop,Chinese Restaurant,Falafel Restaurant,Bank,Convenience Store
9,Crown Heights,Pizza Place,Café,Museum,Bakery,Bagel Shop,Supermarket,Salon / Barbershop,Candy Store,Liquor Store,Deli / Bodega
10,East Flatbush,Deli / Bodega,Print Shop,Caribbean Restaurant,Hardware Store,Park,Chinese Restaurant,Pharmacy,Supermarket,Moving Target,Department Store


In [68]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 4, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
52,Ocean Parkway,Sushi Restaurant,Grocery Store,Gym,Spa,Gym / Fitness Center,Playground,Donut Shop,Paper / Office Supplies Store,Restaurant,Sake Bar


## Results

To define the best possible neighborhood to open a restaurant such as Bills’ Chicago that serves comfort food such as burgers and crafted beers, we need to look at the most common venues within the neighborhoods. Here is a summary of our findings;
<ol>
    <li>Chinese Restaurant is the most common venue in Bensonhurst.</li>
    <li>Mexican Restaurant is the most common venue in Flatbush. </li>
    <li>Italian Restaurant is the most common venue in Carrol Gardens. </li> 
    <li>Fast Food Resturants are common in Cypress Hills</li>
    <li>Prospect Park South Caribbean Restaurant. </li>
    <li>Sushi Restaurants in Ocean Parkway. </li>
    <li>Prospect Park has the biggest Resturant venues in Brooklyn. </li>
    <li>Out of the top 10 venue categories Prospect Park has the most Restaurant category followed by Cypress Hills. </li>
    <li>Cobble Hill has the most comfort food category with Pizza Stores, American Restaurants, Cocktail Bars. </li>
    <li>Possible Neighborhoods that aligns with Bill’s Chicago concept are Greenpoint, Cobble Hill, Cypress Hill and Prospect Park. </li>

## Conclusion

When we look at the Neighborhoods in Brooklyn we see commonalities on the category types for certain areas. Considering the type of food and current branding of Bill's Chicago restaurant by analyzing each neighborhoods venue categories, defining top 5 and 10 most common venues within the neighborhood we think opening a Bills Chicago Burger Restaurant in one of these three neighborhoods will ensure Be Our Guest company profitablity and revenu generation. 
<ol>
    <li>Prospect ParK</li>
    <li>Cypress Hill</li>
    <li>Cobble Hill</li>
</ol>

Our first recommendation is Prospect Park area which we think based on our analysis both burger junkies and locals would love freshly made burgers shakes and fries.